## Random Forest

**For Table 3 of the paper**

Column-based NARVAL R2B4 model

n_estimators = 1 takes 14min 19s

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import os
import sys

from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

base_path = '/home/b/b309170'

import matplotlib.pyplot as plt
import time
import gc

In [2]:
data_path = os.path.join(base_path,
                         'my_work/icon-ml_data/cloud_cover_parameterization/grid_column_based/based_on_var_interpolated_data')

# Load data
input_train = np.load(os.path.join(data_path, 'cloud_cover_input_train_1.npy'))
output_train = np.load(os.path.join(data_path, 'cloud_cover_output_train_1.npy'))
input_valid = np.load(os.path.join(data_path, 'cloud_cover_input_valid_1.npy'))
output_valid = np.load(os.path.join(data_path, 'cloud_cover_output_valid_1.npy'))
input_test = np.load(os.path.join(data_path, 'cloud_cover_input_test_1.npy'))
output_test = np.load(os.path.join(data_path, 'cloud_cover_output_test_1.npy'))

### Random Forest

In [10]:
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 140, random_state = 10)

# Train the model on training data
rf.fit(input_train, output_train)

CPU times: user 14min 18s, sys: 909 ms, total: 14min 19s
Wall time: 14min 19s


RandomForestRegressor(n_estimators=1, n_jobs=-1, random_state=10)

In [4]:
# model_fold_3 is implemented in ICON-A
batch_size = 2**20

for i in range(1 + input_test.shape[0]//batch_size):
    if i == 0:
        clc_predictions = rf.predict(input_test[i*batch_size:(i+1)*batch_size])
    else:
        clc_predictions = np.concatenate((clc_predictions, rf.predict(input_test[i*batch_size:(i+1)*batch_size])), axis=0)
    K.clear_session()
    gc.collect()

In [5]:
mse_rf = mean_squared_error(output_test, clc_predictions)

with open('/home/b/b309170/workspace_icon-ml/iconml_clc/additional_content/baselines/RFs/RF_results.txt', 'a') as file:
    file.write('The MSE on the test set of the column-based R2B4 RF is %.2f.\n'%mse_rf)